# Cache
Copyright (c) Microsoft Corporation. All rights reserved.<br>
Licensed under the MIT License.

A Dataflow can be cached as a file on your disk during a local run by calling `dflow_cached = dflow.cache(directory_path)`. Doing this will run all the steps in the Dataflow, `dflow`, and save the cached data to the specified `directory_path`. The returned Dataflow, `dflow_cached`, has a Caching Step added at the end. Any subsequent runs on on the Dataflow `dflow_cached` will reuse the cached data, and the steps before the Caching Step will not be run again.

Caching avoids running transforms multiple times, which can make local runs more efficient. Here are common places to use Caching:
- after reading data from remote
- after expensive transforms, such as Sort
- after transforms that change the shape of data, such as Sampling, Filter and Summarize

Caching Step will be ignored during scale-out run invoked by `to_spark_dataframe()`.

We will start by reading in a dataset and applying some transforms to the Dataflow.

In [1]:
import azureml.dataprep as dprep
dflow = dprep.read_csv(path='../data/crime-spring.csv')
dflow = dflow.take_sample(probability=0.2, seed=7)
dflow = dflow.sort_asc(columns='Primary Type')
dflow = dflow.keep_columns(['ID', 'Case Number', 'Date', 'Primary Type'])
dflow.head(5)

,ID,Case Number,Date,Primary Type
0,10534446,HZ277630,4/15/2016 10:00,THEFT


Next, we will choose a directory to store the cached data.

In [2]:
import os
from pathlib import Path
cache_dir = str(Path(os.getcwd(), 'dataflow-cache'))
cache_dir

'/mnt/vsts/4/s/target/Python/debug/azureml-dataprep/docs/how-to-guides/dataflow-cache'

We will now call `dflow.cache(directory_path)` to cache the Dataflow to your directory.

In [3]:
dflow_cached = dflow.cache(directory_path=cache_dir)

Now we will check steps in the `dflow_cached` to see that all of the previous steps were cached.

In [4]:
[s.step_type for s in dflow_cached._get_steps()]

['Microsoft.DPrep.GetFilesBlock',
 'Microsoft.DPrep.ParseDelimitedBlock',
 'Microsoft.DPrep.DropColumnsBlock',
 'Microsoft.DPrep.TakeSampleBlock',
 'Microsoft.DPrep.SortBlock',
 'Microsoft.DPrep.KeepColumnsBlock',
 'Microsoft.DPrep.CacheBlock']

We also check the data stored in the cache directory.

In [5]:
os.listdir(cache_dir)

['873d160b-cf61-40e0-baaa-c2073e20de09.cacheIndex',
 'a9fd469a-9909-4e4b-b819-db6031347dea']

Running against `dflow_cached` will reuse the cached data and skip running all of the previous steps again.

In [6]:
dflow_cached.head(5)

,ID,Case Number,Date,Primary Type
0,10534446,HZ277630,4/15/2016 10:00,THEFT


Adding additional steps to `dflow_cached` will also reuse the cache data and skip running the steps prior to the Cache Step.

In [7]:
dflow_cached_take = dflow_cached.take(10)
dflow_cached_skip = dflow_cached.skip(10).take(10)

df_cached_take = dflow_cached_take.to_pandas_dataframe()
df_cached_skip = dflow_cached_skip.to_pandas_dataframe()

In [8]:
# shutil.rmtree will then clean up the cached data 
import shutil
shutil.rmtree(path=cache_dir)